In [ ]:
import pandas as pd
import numpy as np

df= pd.read_csv("../data/qatar2025_f1_data.csv")

In [ ]:
df.head()

In [ ]:
df["Driver"] = df["Driver"].replace({
    "Franco Colapintop": "Franco Colapinto"
})

In [ ]:
clean_df = df.sort_values(
    by=df.columns.tolist(), 
    ascending=False
).drop_duplicates(subset=["Driver"], keep="first").reset_index(drop=True)

In [ ]:
clean_df["Driver"].value_counts()

In [ ]:
season_features_2025 = clean_df.copy()

season_features_2025 = season_features_2025.drop(columns=[
    "QatarLapTime_x",
    "QatarLapTime_y",
    "QatarLap_2021",
    "QatarLap_2023"
], errors="ignore")

season_features_2025.head()

In [ ]:
columns = [
    "AvgQualiPos_2025",
    "AvgRacePos_2025",
    "AvgSprintQualiPos_2025",
    "AvgSprintPos_2025"
]

for col in columns:
    clean_df[col] = clean_df[col].fillna(clean_df[col].mean())

In [ ]:
for col in columns:
    mean = clean_df[col].mean()
    std = clean_df[col].std()
    clean_df[col + "_z"] = (clean_df[col] - mean) / std

In [ ]:
for col in columns:
    clean_df[col + "_z_inv"] = -clean_df[col + "_z"]

In [ ]:
qatar_mean = clean_df["QatarLapTime"].mean()
qatar_std = clean_df["QatarLapTime"].std()

In [ ]:
clean_df["QatarLapTime_z"] = (clean_df["QatarLapTime"] - qatar_mean) / qatar_std

In [ ]:
clean_df["QatarPace_z_inv"] = -clean_df["QatarLapTime_z"]

In [ ]:
w_quali = 1.0
w_race = 1.0
w_sprint_quali = 0.7
w_sprint = 0.7
w_qatar = 1.3 

In [ ]:
clean_df["QatarPace_z_inv_filled"] = clean_df["QatarPace_z_inv"].fillna(0.0)

In [ ]:
clean_df["PerformanceScore"] = (
    w_quali        * clean_df["AvgQualiPos_2025_z_inv"] +
    w_race         * clean_df["AvgRacePos_2025_z_inv"] +
    w_sprint_quali * clean_df["AvgSprintQualiPos_2025_z_inv"] +
    w_sprint       * clean_df["AvgSprintPos_2025_z_inv"] +
    w_qatar        * clean_df["QatarPace_z_inv_filled"]
)

In [ ]:
ranking = (
    clean_df.sort_values("PerformanceScore", ascending=False)
      .reset_index(drop=True)
)

ranking[["Driver", "Team", "PerformanceScore"]].head(15)

In [ ]:
team_view = (
    ranking[["Team", "Driver", "PerformanceScore"]]
    .sort_values(["Team", "PerformanceScore"], ascending=[True, False])
)

team_view["TeamRank"] = team_view.groupby("Team")["PerformanceScore"].rank(ascending=False, method="first")

team_view

In [ ]:
team_strength = team_view.groupby("Team")["PerformanceScore"].mean().sort_values(ascending=False)

team_strength

In [ ]:
rookies = ["Kimi Antonelli", "Oliver Bearman", "Isack Hadjar", "Jack Doohan", "Franco Colapinto", "Gabriel Bortoleto"]

rookie_view = ranking[ranking["Driver"].isin(rookies)][["Driver", "Team", "PerformanceScore"]]
rookie_view